# Kitchen Companies Part I

In [1]:
# import packages
import gurobipy as gp
from gurobipy import GRB
import sensitivity_analysis as sa

In [2]:
# Set up the data
prod_cities = ['bethesda', 'memphis', 'kansas_city']
demand_cities = ['atlanta', 'boston', 'chicago']

capacity = {'bethesda': 18,
            'memphis': 22,
            'kansas_city': 31}
demand = {'atlanta': 10,
          'boston': 8,
          'chicago': 14}

costs = {
    'bethesda': [1675, 400, 685],
    'memphis': [380, 1355, 543],
    'kansas_city': [922, 1646, 700] }

In [3]:
# create the model
kc_1 = gp.Model('kitchen_companies_part1')
kc_1.ModelSense = GRB.MINIMIZE

Restricted license - for non-production use only - expires 2026-11-23


In [4]:
# Create decision variables
dvars = kc_1.addVars(prod_cities, demand_cities, lb=0.0, vtype=GRB.CONTINUOUS, name='x')
kc_1.update()
dvars

{('bethesda', 'atlanta'): <gurobi.Var x[bethesda,atlanta]>,
 ('bethesda', 'boston'): <gurobi.Var x[bethesda,boston]>,
 ('bethesda', 'chicago'): <gurobi.Var x[bethesda,chicago]>,
 ('memphis', 'atlanta'): <gurobi.Var x[memphis,atlanta]>,
 ('memphis', 'boston'): <gurobi.Var x[memphis,boston]>,
 ('memphis', 'chicago'): <gurobi.Var x[memphis,chicago]>,
 ('kansas_city', 'atlanta'): <gurobi.Var x[kansas_city,atlanta]>,
 ('kansas_city', 'boston'): <gurobi.Var x[kansas_city,boston]>,
 ('kansas_city', 'chicago'): <gurobi.Var x[kansas_city,chicago]>}

In [5]:
# Set the objective funtion
kc_1.setObjective(gp.quicksum(costs[prod_city][j]*dvars[(prod_city,demand_city)]
                              for prod_city in prod_cities
                              for j, demand_city in enumerate(demand_cities)))
kc_1.update()
kc_1.display()

Minimize
1675.0 x[bethesda,atlanta] + 400.0 x[bethesda,boston] + 685.0 x[bethesda,chicago]
+ 380.0 x[memphis,atlanta] + 1355.0 x[memphis,boston] + 543.0 x[memphis,chicago]
+ 922.0 x[kansas_city,atlanta] + 1646.0 x[kansas_city,boston]
+ 700.0 x[kansas_city,chicago]
Subject To


C:\Users\zungz\AppData\Local\Temp\ipykernel_7904\1582824989.py:6: DeprecationWarning: Model.display() is deprecated
  kc_1.display()


In [6]:
kc_1.printStats()

Statistics for model 'kitchen_companies_part1':
  Problem type                : LP
  Linear constraint matrix    : 0 rows, 9 columns, 0 nonzeros
  Variable types              : 9 continuous, 0 integer (0 binary)
  Matrix range                : [0e+00, 0e+00]
  Objective range             : [4e+02, 2e+03]
  Bounds range                : [0e+00, 0e+00]
  RHS range                   : [0e+00, 0e+00]


In [7]:
# Create constraints
# Create demand constraints
for demand_city in demand_cities:
    kc_1.addLConstr(gp.quicksum(dvars[(prod_city,demand_city)] for prod_city in prod_cities),
                    GRB.GREATER_EQUAL,
                    demand[demand_city],
                    name=f'demand_{demand_city}')

# Create capacity constraints
for prod_city in prod_cities:
    kc_1.addLConstr(gp.quicksum(dvars[(prod_city,demand_city)] for demand_city in demand_cities),
                    GRB.LESS_EQUAL,
                    capacity[prod_city],
                    name=f'capacity_{prod_city}')

kc_1.update()
kc_1.display()

Minimize
1675.0 x[bethesda,atlanta] + 400.0 x[bethesda,boston] + 685.0 x[bethesda,chicago]
+ 380.0 x[memphis,atlanta] + 1355.0 x[memphis,boston] + 543.0 x[memphis,chicago]
+ 922.0 x[kansas_city,atlanta] + 1646.0 x[kansas_city,boston]
+ 700.0 x[kansas_city,chicago]
Subject To
  demand_atlanta: x[bethesda,atlanta] + x[memphis,atlanta] + x[kansas_city,atlanta] >= 10
  demand_boston: x[bethesda,boston] + x[memphis,boston] + x[kansas_city,boston] >= 8
  demand_chicago: x[bethesda,chicago] + x[memphis,chicago] + x[kansas_city,chicago] >= 14
  capacity_bethesda: x[bethesda,atlanta] + x[bethesda,boston] + x[bethesda,chicago] <= 18
  capacity_memphis: x[memphis,atlanta] + x[memphis,boston] + x[memphis,chicago] <= 22
capacity_kansas_city: x[kansas_city,atlanta] + x[kansas_city,boston] +
 x[kansas_city,chicago] <= 31


C:\Users\zungz\AppData\Local\Temp\ipykernel_7904\839405479.py:17: DeprecationWarning: Model.display() is deprecated
  kc_1.display()


In [8]:
# optimize
kc_1.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 8945HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0x44c3f6e8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+02, 2e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 3e+01]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.200000e+01   0.000000e+00      0s
       4    1.4886000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.488600000e+04


In [9]:
sa.sa_vars(kc_1.getVars())

,final_value,reduced_cost,obj_coef,range_opt_low,range_opt_high
"x[bethesda,atlanta]",0.0,1153.0,1675.0,522.0,inf
"x[bethesda,boston]",8.0,0.0,400.0,0.0,1497.0
"x[bethesda,chicago]",2.0,0.0,685.0,543.0,700.0
"x[memphis,atlanta]",10.0,0.0,380.0,-142.0,780.0
"x[memphis,boston]",0.0,1097.0,1355.0,258.0,inf
"x[memphis,chicago]",12.0,0.0,543.0,143.0,685.0
"x[kansas_city,atlanta]",0.0,400.0,922.0,522.0,inf
"x[kansas_city,boston]",0.0,1246.0,1646.0,400.0,inf
"x[kansas_city,chicago]",0.0,15.0,700.0,685.0,inf


In [10]:
sa.sa_constrs(kc_1.getConstrs())

,binding?,final_value,RHS,slack,shadow_price,range_feas_low,range_feas_high
demand_atlanta,binding,10.0,10.0,0.0,522.0,8.0,18.0
demand_boston,binding,8.0,8.0,0.0,400.0,-0.0,16.0
demand_chicago,binding,14.0,14.0,0.0,685.0,12.0,22.0
capacity_bethesda,non-binding,10.0,18.0,8.0,0.0,10.0,inf
capacity_memphis,binding,22.0,22.0,0.0,-142.0,14.0,24.0
capacity_kansas_city,non-binding,0.0,31.0,31.0,0.0,0.0,inf
